# Refinitiv Data Platform Library for Python
## Content - Pricing - Streaming

This notebook demonstrates how a Price object to receive streaming real-time data.  

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook - which I have done, and so I don't need to specify a session_type:

In [6]:
# I have defined a default Session type in the Credentials notebook - so no need to provide one here
open_session()

send: b'\x81\x90n\xf4\x98\x9a\x15\xd6\xcc\xe3\x1e\x91\xba\xa0N\xd6\xc8\xf5\x00\x93\xba\xe7'
send: b'\x81\x90\x9e&\xf3\x15\xe5\x04\xa7l\xeeC\xd1/\xbe\x04\xa3z\xf0A\xd1h'
send: b'\x81\x90)\xc7\xcf_R\xe5\x9b&Y\xa2\xede\t\xe5\x9f0G\xa0\xed"'
send: b'\x81\x90\xb3TO\x8b\xc8v\x1b\xf2\xc31m\xb1\x93v\x1f\xe4\xdd3m\xf6'
send: b'\x81\x90\x05\xcbU\xd9~\xe9\x01\xa0u\xaew\xe3%\xe9\x05\xb6k\xacw\xa4'
send: b'\x81\x90\x16v\xeaUmT\xbe,f\x13\xc8o6T\xba:x\x11\xc8('
send: b'\x81\xfe\x11\xc5hd\x0f\t\x13FKf\x05\x05fgJ^/+$\x0bh`\x06F#)J-K+RD=%HFDl\x11F5)\x13FAh\x05\x01[p\x18\x01-3HFN|\x1c\x0ca]\x07\x0fjgJH/+-\x08jd\r\n{zJ^/rJ%z}\x00\x01a}\x01\x07n}\x01\x0ba]\x07\x0fjgJ^/+\r\x1dE9\r<N`\'\rEa\x0c\'{x\x0cW^`$\'Ea\n#l`\'\rEZ=\x1eF8&\rFz!\t{y2\'F?!\t]D\x0c"k;9T{J;\'>G\x0b\x08Yp?\t6P%<If\x0cV\x7fS9V{8\r Y9>VUZ;V`=%!6\x7f\x0b\x0ek\\1V7`\x0e5!l\x11.dP06g@\x02\x0bflY\x13fPZ\x08xh/2vm/2;m.\x13fF\x04\x13f[\x12\x0c~j\x12&\x7fm[6LhY&|h?\x00vS \x14W[\x12\x00gmZ>US\x066dF/\x1c@h\x04-{QZ\x00DX\x04\x00>].&L[<.JX[6UZ0.fm\x046

### Define a callback to receive streaming data

In [7]:
# declare a DataFrame to use for storing our streaming data
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

## Create a stream and open it

In [8]:
# Define our Streaming Price object
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# Specify callback handler for any updates
streams.on_update(update_dataframe)
# Callback for when we have received initial values for all items
streams.on_complete(create_dataframe)
# Send the requests to the server and open the streams for all items
streams.open()

,BID,ASK
EUR=,1.2099,1.2101
GBP=,1.4114,1.4116
JPY=,109.75,109.76
CAD=,1.2159,1.216


## Close the stream

In [9]:
streams.close()

send: b'\x81\x9aG\x9d\xefC<\xbf\xa6\x07e\xa7\xcfuk\xbd\xcd\x17>\xed\x8aa}\xbd\xcd\x00+\xf2\x9c&e\xe0'
send: b"\x81\x9a\x84\x07)u\xff%`1\xa6=\tB\xa8'\x0b!\xfdwLW\xbe'\x0b6\xe8hZ\x10\xa6z"
send: b'\x81\x9a\x90\xd2x\xe8\xeb\xf01\xac\xb2\xe8X\xd0\xbc\xf2Z\xbc\xe9\xa2\x1d\xca\xaa\xf2Z\xab\xfc\xbd\x0b\x8d\xb2\xaf'
send: b'\x81\x9a\x93\xdcw\xe9\xe8\xfe>\xad\xb1\xe6W\xd0\xbf\xfcU\xbd\xea\xac\x12\xcb\xa9\xfcU\xaa\xff\xb3\x04\x8c\xb1\xa1'


<StreamState.Closed: 1>

## Close the session

In [10]:
close_session()

send: b'\x81\xad\xd6#\xaa"\xad\x01\xeeM\xbbB\xc3L\xf4\x19\x8a\x00\x9aL\xcdK\xb8\x01\x86\x02\xf4j\xee\x00\xec\x03\x98\x0e\xf6\x01\xfe[\xa6F\x88\x18\xf6\x01\xe9N\xb9P\xcf\x00\xab'
send: b'\x88\x82\xcb\xd1\xb0\xbc\xc89'
